# Apply SVM

In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(e1071)

In [2]:
set.seed(1)

In [3]:
# invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'TimeHeadway')
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'RightLamp', 'LeftLamp', 'UpLamp')

In [4]:
valuables <- c("RoadType","CurveAverage","Speed","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed","LaneCount","Engine","PathType","SteeringAngle")

In [5]:
printf <- function(...) cat(sprintf(...))

In [6]:
CV <- function(dfx, valuables, verbose=FALSE) {        
    folds <- createFolds(dfx$flag)
    
    count <- 1
    errs <- c()
    reds <- c()
    blues <- c()
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]         
        fit <- svm(x=train[, valuables], y=train[, "flag"], data=train, type='C-classification', kernel='radial', gamma=0.3162278, cost=0.6309573)
        # fit <- svm(x=train[, valuables], y=train[, "flag"], data=train, type='C-classification', kernel='radial', gamma=0.1, cost=1)
        p <- predict(fit, newdata=test[, valuables])
        
        if (verbose) {            
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, p)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        }
        
        nerr <- sum((p == test$flag) == FALSE)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(c(mean(errs), mean(reds), mean(blues)))
}

In [7]:
correctVsPredict <- function(test, predictedFlags) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum((predictedRedRows$flag == 'Red') == TRUE)
    nPredictedReds <- nrow(predictedRedRows)
    printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum((predictedBlueRows$flag == 'Blue') == TRUE)
    nPredictedBlues <- nrow(predictedBlueRows)
    printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [8]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [9]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Red

In [10]:
df3 <- read.csv("../data/middle/sp4.csv", stringsAsFactors=FALSE)

In [11]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [12]:
set.seed(1)

In [13]:
result <- CV(df3, valuables, verbose=TRUE)
printRedRatios(df3)
printf("Red: Mean correct/predict = %f\n", result[2])
printf("Blue: Mean correct/predict = %f\n", result[3])
printf("CV value: %f", result[1])

Fold1
As for Red: correct/predict = 49/70 = 0.700000
As for Blue: correct/predict = 23/34 = 0.676471

Fold2
As for Red: correct/predict = 51/80 = 0.637500
As for Blue: correct/predict = 16/25 = 0.640000

Fold3
As for Red: correct/predict = 52/87 = 0.597701
As for Blue: correct/predict = 10/17 = 0.588235

Fold4
As for Red: correct/predict = 44/70 = 0.628571
As for Blue: correct/predict = 19/34 = 0.558824

Fold5
As for Red: correct/predict = 51/80 = 0.637500
As for Blue: correct/predict = 16/24 = 0.666667

Fold6
As for Red: correct/predict = 50/81 = 0.617284
As for Blue: correct/predict = 14/23 = 0.608696

Fold7
As for Red: correct/predict = 45/67 = 0.671642
As for Blue: correct/predict = 23/38 = 0.605263

Fold8
As for Red: correct/predict = 54/80 = 0.675000
As for Blue: correct/predict = 19/24 = 0.791667

Fold9
As for Red: correct/predict = 50/80 = 0.625000
As for Blue: correct/predict = 15/24 = 0.625000

Fold10
As for Red: correct/predict = 49/76 = 0.644737
As for Blue: correct/predict

In [14]:
dfx <- df3[, c(valuables, "flag")]

In [16]:
# gamma.lim <- 10^(seq(-5, 5, 0.1))
cost.lim <- 10^(seq(-2, 2, 0.1))

tune <- tune.svm(
        flag ~ .,
        data=dfx,
        kernel='radial',
        gamma = gamma.lim,
        cost = cost.lim,
        tunecontrol = tune.control(sampling = "cross", cross = 10)
)

In [ ]:
# summary(tune)

Parameter tuning of ‘svm’:

- sampling method: 5-fold cross validation   

- best parameters:   
     gamma      cost   
 0.3162278 0.6309573   
  
- best performance: 0.3522037   
